In [10]:
# %pip install beautifulsoup4

In [11]:
import requests
from bs4 import BeautifulSoup

In [12]:
url = "https://display.musinsa.com/display/api/v2/category-menu?gf=A"
res = requests.get(url)
res.status_code

200

In [13]:
musinsa_data = res.json()['data']['menuItems']

In [14]:
# 카테고리 분류 
category_point_url = []

# len(musinsa_data) --> 5로 지정한 이유 : [상의, 아우터, 바지]까지만 수집
for i in range(3): 
    menugroups = musinsa_data[i]['menuGroups'][0]['menuItems']

     # 2부터 시작하는 이유 : [** 전체, ** 신상] 제외, 카테고리 세분화를 위해서 설정
    for j in range(2,len(menugroups)):
        category_point_url.append((menugroups[j]['title'], menugroups[j]['landingUrl'].split('/')[-1]))

# len(musinsa_data) --> 4, 5로 지정한 이유 : [원피스/스커트, 신발] 수집
for i in range(3, 5):
    for k in range(2):
        menugroups = musinsa_data[i]['menuGroups'][k]['menuItems']

        # 2부터 시작하는 이유 : [** 전체, ** 신상] 제외, 카테고리 세분화를 위해서 설정
        for j in range(2,len(menugroups)):
            category_point_url.append((menugroups[j]['title'], menugroups[j]['landingUrl'].split('/')[-1]))

In [15]:
print(category_point_url)
print(category_point_url[0][1], len(category_point_url))

[('맨투맨/스웨트셔츠', '001005'), ('셔츠/블라우스', '001002'), ('후드 티셔츠', '001004'), ('니트/스웨터', '001006'), ('피케/카라 티셔츠', '001003'), ('긴소매 티셔츠', '001010'), ('반소매 티셔츠', '001001'), ('민소매 티셔츠', '001011'), ('스포츠 상의', '001013'), ('기타 상의', '001008'), ('후드 집업', '002022'), ('블루종/MA-1', '002001'), ('레더/라이더스 재킷', '002002'), ('무스탕/퍼', '002025'), ('트러커 재킷', '002017'), ('슈트/블레이저 재킷', '002003'), ('카디건', '002020'), ('아노락 재킷', '002019'), ('플리스/뽀글이', '002023'), ('트레이닝 재킷', '002018'), ('스타디움 재킷', '002004'), ('환절기 코트', '002008'), ('겨울 싱글 코트', '002007'), ('겨울 더블 코트', '002024'), ('겨울 기타 코트', '002009'), ('롱패딩/롱헤비 아우터', '002013'), ('숏패딩/숏헤비 아우터', '002012'), ('패딩 베스트', '002016'), ('베스트', '002021'), ('사파리/헌팅 재킷', '002014'), ('나일론/코치 재킷', '002006'), ('기타 아우터', '002015'), ('데님 팬츠', '003002'), ('코튼 팬츠', '003007'), ('슈트 팬츠/슬랙스', '003008'), ('트레이닝/조거 팬츠', '003004'), ('숏 팬츠', '003009'), ('레깅스', '003005'), ('점프 슈트/오버올', '003010'), ('스포츠 하의', '003011'), ('기타 바지', '003006'), ('미니 원피스', '020006'), ('미디 원피스', '020007'), ('맥시 원피스', '020

In [16]:
# # 카테고리 별 옵션정보 확인용
# url = [f"https://display.musinsa.com/display/api/v2/categories/ITEM/goods/filter/detail?siteKindId=musinsa&sex=A&originalYn=N&sendLog=true&kidsYn=N&categoryCode={category_point_url[i][1]}" for i in range(len(category_point_url))]
# url, len(url)

In [17]:
json_data = requests.get(f'https://display.musinsa.com/display/api/v2/categories/ITEM/goods/filter/detail?siteKindId=musinsa&sex=A&originalYn=N&sendLog=true&kidsYn=N&categoryCode=001005').json()
json_data['data'][1]

{'key': 'attribute_11',
 'displayText': '소매기장',
 'filterContents': [{'code': '11^224',
   'name': '민소매',
   'position': 'goodsAttributes'},
  {'code': '11^225', 'name': '반소매', 'position': 'goodsAttributes'},
  {'code': '11^227', 'name': '3/4길이', 'position': 'goodsAttributes'},
  {'code': '11^228', 'name': '긴소매', 'position': 'goodsAttributes'}]}

In [18]:
total_dict = {}
for i in range(len(category_point_url)):
    category_name, category_code = category_point_url[i]
    json_data = requests.get(f'https://display.musinsa.com/display/api/v2/categories/ITEM/goods/filter/detail?siteKindId=musinsa&sex=A&originalYn=N&sendLog=true&kidsYn=N&categoryCode={category_code}').json()
    category_data = [
        {
            (json_data['data'][j]['key'] , json_data['data'][j]['displayText']): [
                name for name in [json_data['data'][j]['filterContents'][k]['name']
                for k in range(len(json_data['data'][j]['filterContents']))]
                if name != ''
            ]
        }
        for j in range(len(json_data['data']))
    ]

    # 빈 리스트 요소 및 빈 문자열 제거 후 다시 필터링
    category_data = [item for item in category_data if list(item.values())[0]]

    total_dict[category_point_url[i]] = category_data

# 출력 결과 확인
total_dict

{('맨투맨/스웨트셔츠',
  '001005'): [{('attribute_11', '소매기장'): ['민소매',
    '반소매',
    '3/4길이',
    '긴소매']}, {('attribute_2', '핏'): ['슬림', '레귤러', '오버사이즈']}, {('attribute_1',
    '주요소재'): ['면',
    '폴리에스테르',
    '기모',
    '나일론',
    '폴리우레탄',
    '스판덱스',
    '레이온/인견',
    '울',
    '아크릴',
    '플리스',
    '니트',
    '텐셀',
    '모달',
    '코듀로이',
    '코마사']}, {('attribute_132', '주요장식'): ['니트패널',
    '디스트로이드',
    '라인',
    '레이어드',
    '셔링',
    '스트링',
    '슬릿',
    '언발란스',
    '컷아웃',
    '턱',
    '프릴',
    '플리츠']}, {('attribute_3', '넥라인'): ['U넥',
    'V넥',
    '헨리넥',
    '터틀넥',
    '하프넥',
    '일자탑',
    '라운드넥',
    '스퀘어넥',
    '보트넥',
    '하트넥',
    '리본넥',
    '후드']}, {('discountRate', '할인율'): ['30% 미만',
    '30%~50% 미만',
    '50%~70% 미만',
    '70% 이상']}, {('price', '가격'): ['5만원 이하',
    '5만원 ~ 10만원',
    '10만원 ~ 20만원',
    '20만원 ~ 30만원',
    '30만원 이상']}, {('color', '색상'): ['블랙',
    '라이트 그레이',
    '그레이',
    '네이비',
    '아이보리',
    '화이트',
    '다크 그레이',
    '블루',
    '그린',
    '스카이 블루',
    '베이지',
    '기

In [ ]:
# %pip install pandas

In [8]:
import pandas as pd

In [23]:
# 1. Creating the main and sub category tables

# Extracting main and sub categories
main_category_data = []
sub_category_data = []

for (category, subcategory), _ in total_dict.items():
    main_category = subcategory[:3]
    sub_category = subcategory[3:]
    if main_category == '001':
        main_category_data.append([main_category, '상의'])
    elif main_category == '002':
        main_category_data.append([main_category, '아우터'])
    elif main_category == '003':
        main_category_data.append([main_category, '바지'])
    elif main_category == '020':
        main_category_data.append([main_category, '원피스'])
    elif main_category == '022':
        main_category_data.append([main_category, '스커트'])
    elif main_category == '018':
        main_category_data.append([main_category, '스니커즈'])
    elif main_category == '005':
        main_category_data.append([main_category, '신발'])
    sub_category_data.append([sub_category, main_category, category])

df_main = pd.DataFrame(main_category_data, columns=['main_cat_id', 'main_cat_name']).drop_duplicates()
df_sub = pd.DataFrame(sub_category_data, columns=['sub_cat_id', 'main_cat_id', 'sub_cat_name']).drop_duplicates()

# 2. Creating the option table

option_data = []
option_id = 1
option_lookup = {}

for (category, subcategory), attributes in total_dict.items():
    for attribute_dict in attributes:
        for (attr_code, attr_name), attr_values in attribute_dict.items():
            for value in attr_values:
                key = (attr_code, value)
                if key not in option_lookup:
                    option_lookup[key] = option_id
                    option_data.append([option_id, attr_name, value])
                    option_id += 1

df_option = pd.DataFrame(option_data, columns=['option_cat_id', 'option_big_name', 'option_small_name'])

# 3. Creating the sub-option table

sub_option_data = []
sub_option_id = 1

for (category, subcategory), attributes in total_dict.items():
    main_category = subcategory[:3]
    sub_category = subcategory[3:]
    for attribute_dict in attributes:
        for (attr_code, attr_name), attr_values in attribute_dict.items():
            for value in attr_values:
                option_id = option_lookup[(attr_code, value)]
                sub_option_data.append([sub_option_id, main_category, sub_category, option_id])
                sub_option_id += 1

df_sub_option = pd.DataFrame(sub_option_data, columns=['sub_option_id', 'main_cat_id', 'sub_cat_id', 'option_cat_id'])

# 4. Creating the detail table (with sample product data)

detail_data = [
    {"prod_id": 1, "prod_name": "상품 1", "prod_price": 10000, "prod_url": "https://...", "prod_img": "img1.jpg", "sub_option_id": 1},
    {"prod_id": 2, "prod_name": "상품 2", "prod_price": 20000, "prod_url": "https://...", "prod_img": "img2.jpg", "sub_option_id": 2},
    {"prod_id": 3, "prod_name": "상품 3", "prod_price": 30000, "prod_url": "https://...", "prod_img": "img3.jpg", "sub_option_id": 3},
]

df_detail = pd.DataFrame(detail_data)

df_main, df_sub, df_option, df_sub_option, df_detail

(   main_cat_id main_cat_name
 0          001            상의
 10         002           아우터
 32         003            바지
 41         020           원피스
 44         022           스커트
 47         018          스니커즈
 51         005            신발,
    sub_cat_id main_cat_id sub_cat_name
 0         005         001    맨투맨/스웨트셔츠
 1         002         001      셔츠/블라우스
 2         004         001       후드 티셔츠
 3         006         001       니트/스웨터
 4         003         001    피케/카라 티셔츠
 ..        ...         ...          ...
 57        018         005          슬리퍼
 58        006         005        기타 신발
 59        016         005    모카신/보트 슈즈
 60        011         005           부츠
 61        005         005        신발 용품
 
 [62 rows x 3 columns],
      option_cat_id option_big_name option_small_name
 0                1            소매기장               민소매
 1                2            소매기장               반소매
 2                3            소매기장             3/4길이
 3                4            소매기장  

In [ ]:
'https://display.musinsa.com/display/api/v2/categories/ITEM/goods?siteKindId=musinsa&sex=A&sortCode=pop_category&colorCodes=10,37&goodsAttributes=2^87&categoryCode=001001&size=60&page=1'

In [8]:
prod_data = requests.get('https://display.musinsa.com/display/api/v2/categories/ITEM/goods?siteKindId=musinsa&sex=A&sortCode=pop_category&categoryCode=001005&size=60&page=1&color=1&goodsattributes=10').json()['data']['goodsList']
prod_data[1]

{'goodsNo': 3174960,
 'subNo': 0,
 'goodsName': '[SET] (긴바지 OR 반바지 선택) 화섬 배색 스포츠 크루넥 셋업_스카이블루',
 'linkUrl': 'https://www.musinsa.com/app/goods/3174960',
 'imageUrl': 'https://image.msscdn.net/thumbnails/images/goods_img/20230323/3174960/3174960_17173833374376_500.jpg',
 'sex': 6,
 'isOptionLayer': True,
 'isSoldOut': False,
 'isComingSoon': False,
 'comingSoonDateFormat': '',
 'relatedGoodsReviewCount': 366,
 'relatedGoodsReviewScore': 96,
 'normalPrice': 122900,
 'price': 89900,
 'saleRate': 27,
 'timeSaleText': '',
 'couponDiscountPrice': 0,
 'brand': 'codegraphy',
 'brandName': '코드그라피',
 'brandNameEng': 'CODEGRAPHY',
 'brandLinkUrl': 'https://www.musinsa.com/brands/codegraphy',
 'goodsLabel': [{'code': 'EXCLUSIVE_LIMIT',
   'badgeTitle': '한정 판매',
   'badgeShortTitle': '한정',
   'badgeClass': '',
   'badgeColor': ''}],
 'displaySexCodeList': ['M', 'W'],
 'deliveryDueType': '',
 'deliveryText': '',
 'codi': None}

In [8]:
page = 1
category_code = list(total_dict.keys())
all_item = {}
for i in range(len(category_code)):
    while True:
        url1 = 'https://display.musinsa.com/display/api/v2/categories/ITEM/goods?siteKindId=musinsa&sex=A&sortCode=pop_category&categoryCode={category_code[i][1]}&size=60&page={page}'
        res1 = requests.get(url1)
        if res1.status_code ==  200:
            prod_data = res1.json()['data']['goodsList']
            item = []
            for j in range(len(prod_data)):
                item_name = prod_data[j]['goodsName']
                item_price = prod_data[j]['price']
                item_image = prod_data[j]['imageUrl']
                item_url= prod_data[j]['linkUrl']
                item.append((item_name, item_price, item_image, item_url))
            all_item[category_code[i]] = item
all_item

KeyboardInterrupt: 

In [37]:
import requests

category_code = list(total_dict.keys())
all_item = {}

for i in range(len(category_code)):
    page = 1  # 각 카테고리마다 페이지를 1로 초기화
    while True:
        url1 = f'https://display.musinsa.com/display/api/v2/categories/ITEM/goods?siteKindId=musinsa&sex=A&sortCode=pop_category&categoryCode={category_code[i][1]}&size=60&page={page}'
        res1 = requests.get(url1)
        if res1.status_code == 200:
            prod_data = res1.json().get('data', {}).get('goodsList', [])
            if not prod_data:
                break  # If no more products are found, break the loop

            item = []
            for j in range(len(prod_data)):
                item_name = prod_data[j]['goodsName']
                item_price = prod_data[j]['price']
                item_image = prod_data[j]['imageUrl']
                item_url = prod_data[j]['linkUrl']
                item.append((item_name, item_price, item_image, item_url))

            # Initialize the list for this category if it does not exist
            if category_code[i] not in all_item:
                all_item[category_code[i]] = []

            all_item[category_code[i]].append(item)
            page += 1
        else:
            break  # If the request was not successful, break the loop

print(all_item)


{}


In [ ]:
# import itertools

# # 하나의 key에 속한 value 리스트 안의 요소들에 대한 조합 생성
# combinations_by_category = {}
# for key, filters in total_dict.items():
#     filter_values = [list(item.values())[0] for item in filters]
#     if filter_values:
#         flattened_values = [val for sublist in filter_values for val in sublist]
#         combinations_by_category[key] = []
#         for r in range(1, len(flattened_values) + 1):
#             combinations_by_category[key].extend(itertools.combinations(flattened_values, r))

In [ ]:
# import itertools

# # 예제 카테고리 리스트
# categories = [
#     ['Red', 'Blue', 'Green'],
#     ['Small', 'Medium', 'Large'],
#     ['Cotton', 'Polyester']
# ]

# # 모든 가능한 카테고리 조합 생성
# all_combinations = []
# for r in range(1, len(categories) + 1):
#     for combo in itertools.combinations(categories, r):
#         all_combinations.extend(itertools.product(*combo))

# # 조합 출력
# for combo in all_combinations:
#     print(combo)



In [ ]:
# %pip install pymysql
# import pymysql